In [21]:
from openai import OpenAI

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import START, StateGraph, END

from IPython.display import Markdown, Image
from typing_extensions import TypedDict

import json
import ast

In [22]:
from dotenv import load_dotenv
load_dotenv()

# create openAI client
client = OpenAI()

In [23]:
# Build a TypedDict Class to store information
class AgentState(TypedDict):
    topic: str
    topic_validity: str
    response: str
    #cont: str
    recursion_limit: int

In [24]:
def greet_user(state):
    system_message = """
    You are Dave Chappelle, the funny comedian.
    you ask if the user wants to hear a joke.
    instruct to answer with a yes or no.
    """

    completion = client.chat.completions.create(
      model="gpt-4.1-mini",
      messages=[
        {"role": "system", "content": system_message}
        ]
      )
    greet = completion.choices[0].message.content
    print(greet)
    user_input = input()
    state['response'] = user_input

    return state

In [28]:
# Define a function that gets and validated the topic
def get_topic(state):
    print("what type of joke are you in the mood for?")
    user_input = input()
    state['topic'] = user_input

    system_message = """
    Validate the user's preference.
    Return as JSON:
    If valid, return {'valid': 'yes', 'response': <a joke related to the preference>}.
    if invalid, return {'valid': 'no', 'response': 'Please enter a valid topic'}.
    """
    completion = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "system", "content": system_message},
              {"role": "user", "content": f"{state['topic']}"}],
    response_format={ "type": "json_object" }
    )

    try:
        bot = json.loads(completion.choices[0].message.content)
    except json.decoder.JSONDecodeError:
        print("Model did not return a valid JSON")

    state['topic_validity'] = bot.get('valid')
    print(bot.get('response'))

    return state


In [27]:
# Function to say goodbye
def goodbye(state):
    print("Goodbye!")
    return state

In [29]:
def response_router(state):
    if state['response'].lower() == "yes":
        return 'Continue'
    elif state['response'].lower() == "no":
        return 'End'

In [30]:
def topic_router(state):
    if state['topic_validity'] == "yes":
        return 'Valid'
    elif state['topic_validity'] == "no":
        return 'Invalid'

In [31]:
def continuation(state):
    print("Do you want another joke? Reply yes or no")
    user_input = input()
    state['response'] = user_input
    return state

In [32]:
# Initializint a StateGraph using the AgentState
workflow = StateGraph(AgentState)

# Set an entry point for the workflow
workflow.set_entry_point("Greet User")

# Define the nodes and their functions
workflow.add_node("Greet User", greet_user)
workflow.add_node("Get Topic & Check Validity", get_topic)
workflow.add_node("Continue?", continuation)
workflow.add_node("Say Bye", goodbye)

# Define a conditional edge
workflow.add_conditional_edges(
    "Greet User",
    response_router,
    {"Continue": "Get Topic & Check Validity",
     "End": "Say Bye"}
)

workflow.add_conditional_edges(
    "Get Topic & Check Validity",
    topic_router,
    {"Valid": "Continue?",
     "Invalid": "Get Topic & Check Validity"}
)

workflow.add_conditional_edges(
    "Continue?",
    response_router,
    {"Continue": "Get Topic & Check Validity",
     "End": "Say Bye"}
)

# Workflow add edge
workflow.add_edge("Say Bye", END)


# Compile the workflow
app = workflow.compile()


In [35]:
# Run the app
result = app.invoke({'recursion_limit': 10})

Hey there! You wanna hear a joke? Just hit me with a "yes" or "no."
what type of Joke are you in the mood for?
Please enter a valid topic
what type of Joke are you in the mood for?
Why did the shoe go to school? Because it wanted to be a "smart sneaker"!
Do you want another joke? Reply yes or no


KeyboardInterrupt: Interrupted by user